# Google Colab A100 - Grass Species Mass Generation (100 images each)

Geração em massa de imagens de espécies de gramíneas otimizada para **Google Colab A100 GPU**.

## Configuração:
- **100 imagens por tipo** (3 espécies × 3 condições = 9 tipos)
- **Total: 900 imagens**
- **Seeds aleatórias** para máxima variabilidade
- **Salvamento automático no Google Drive** com estrutura organizada
- **Otimizado para A100 GPU** com batch processing

## Espécies e Condições:
- **Brachiaria brizantha**: Saúde Ótima → Boa Condição → Estresse Moderado
- **Panicum maximum**: Vigor Excelente → Crescimento Adequado → Vigor em Declínio  
- **Cynodon dactylon**: Condição Premium → Qualidade Padrão → Cobertura Rarefeita

<a href="https://colab.research.google.com/github/Kiwiabacaxi/img-sinth/blob/main/notebooks/generation/Colab_A100_GrassGeneration_100imgs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages for A100 optimization
!pip install diffusers transformers accelerate torch torchvision huggingface_hub xformers --quiet
!pip install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu118 --quiet

In [ ]:
# Import libraries
import torch
from diffusers import StableDiffusion3Pipeline
from huggingface_hub import login
import matplotlib.pyplot as plt
from PIL import Image
import os
import random
import time
import gc
from datetime import datetime
import numpy as np
from tqdm import tqdm

In [ ]:
# Check GPU availability and optimize for A100
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    # A100 specific optimizations
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print("✓ A100 TF32 optimizations enabled")
else:
    print("⚠️ No GPU available - this will be very slow!")

In [ ]:
# Hugging Face login (required for SD 3.5 Medium)
# You need to get a token from https://huggingface.co/settings/tokens
login()

In [ ]:
# Mount Google Drive early to ensure it's available
from google.colab import drive
drive.mount('/content/drive')

# Create base directory structure
base_drive_dir = '/content/drive/MyDrive/grass_mass_generation'
os.makedirs(base_drive_dir, exist_ok=True)
print(f"✓ Google Drive mounted and base directory created: {base_drive_dir}")

In [ ]:
# Load model with A100 optimizations
print("Loading Stable Diffusion 3.5 Medium with A100 optimizations...")

pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    torch_dtype=torch.float16,
    device_map="auto",
    use_safetensors=True
)

# A100 specific optimizations
pipe.enable_model_cpu_offload()
pipe.enable_attention_slicing()

# Try to enable xformers if available
try:
    pipe.enable_xformers_memory_efficient_attention()
    print("✓ XFormers memory efficient attention enabled")
except Exception as e:
    print(f"⚠️ XFormers not available: {e}")

print("✓ Model loaded and optimized for A100!")

In [ ]:
# Define all grass species prompts for mass generation
grass_prompts = {
    # Brachiaria brizantha
    'brachiaria_saude_otima': {
        'positive': (
            "perfect overhead view of prime brachiaria brizantha pasture, "
            "thick vibrant green tropical grass, dense uniform coverage, "
            "wide leaf blades in natural tufts, robust plant structure, "
            "excellent ground coverage, no bare soil visible, "
            "agricultural research photography, natural daylight, "
            "top-down perspective, healthy tropical grassland"
        ),
        'negative': (
            "sparse coverage, thin grass, bare patches, yellow grass, "
            "ground level view, side angle, artificial, synthetic, "
            "ornamental lawn, temperate species, flowers, weeds"
        ),
        'species': 'brachiaria',
        'condition': 'saude_otima'
    },
    
    'brachiaria_boa_condicao': {
        'positive': (
            "aerial view of good condition brachiaria pasture, "
            "mostly green grass with natural color variation, "
            "good coverage with minor irregular patches, "
            "wide leaf tropical grass showing normal field variation, "
            "scientific documentation, natural lighting, "
            "overhead perspective, realistic pasture condition"
        ),
        'negative': (
            "severely damaged, mostly bare soil, dying grass, "
            "ground perspective, ornamental setting, artificial, "
            "temperate grass, decorative lawn, flowers"
        ),
        'species': 'brachiaria',
        'condition': 'boa_condicao'
    },
    
    'brachiaria_estresse_moderado': {
        'positive': (
            "top-down view of brachiaria pasture under moderate stress, "
            "mixed green and lighter colored grass areas, "
            "uneven plant density, some small bare soil spots visible, "
            "tropical grass showing stress but still recognizable, "
            "natural variation in coverage, field research documentation, "
            "realistic pasture with environmental stress"
        ),
        'negative': (
            "completely healthy, uniform green, perfect coverage, "
            "ground level view, decorative grass, artificial, "
            "abstract patterns, unrecognizable vegetation, flowers"
        ),
        'species': 'brachiaria',
        'condition': 'estresse_moderado'
    },
    
    # Panicum maximum
    'panicum_vigor_excelente': {
        'positive': (
            "overhead view of vigorous panicum maximum pasture, "
            "tall robust green tropical grass, wide leaf blades, "
            "dense natural clumping pattern, excellent coverage, "
            "healthy upright growth, thick stems visible, "
            "research quality photography, natural lighting, "
            "top-down agricultural perspective"
        ),
        'negative': (
            "short grass, thin blades, sparse coverage, ground view, "
            "artificial, ornamental, temperate species, flowers, "
            "decorative lawn, synthetic"
        ),
        'species': 'panicum',
        'condition': 'vigor_excelente'
    },
    
    'panicum_crescimento_adequado': {
        'positive': (
            "aerial view of adequate panicum grass field, "
            "mostly green tall grass with natural variation, "
            "good but not perfect coverage, mixed growth vigor, "
            "large tropical grass with normal field irregularities, "
            "scientific documentation, top-down perspective, "
            "realistic pasture conditions"
        ),
        'negative': (
            "severely stressed, mostly bare, dying vegetation, "
            "ground perspective, decorative setting, artificial, "
            "ornamental lawn, flowers, synthetic"
        ),
        'species': 'panicum',
        'condition': 'crescimento_adequado'
    },
    
    'panicum_vigor_declinio': {
        'positive': (
            "overhead documentation of declining panicum pasture, "
            "tall grass with mixed green and pale areas, "
            "reduced plant density, some bare patches, "
            "tropical grass showing stress but maintaining structure, "
            "natural field conditions, research photography, "
            "aerial agricultural documentation"
        ),
        'negative': (
            "perfect health, uniform coverage, decorative lawn, "
            "ground view, abstract patterns, unrecognizable, "
            "artificial, ornamental, flowers"
        ),
        'species': 'panicum',
        'condition': 'vigor_declinio'
    },
    
    # Cynodon dactylon
    'cynodon_condicao_premium': {
        'positive': (
            "perfect overhead view of premium cynodon dactylon, "
            "dense carpet-like grass coverage, uniform bright green, "
            "fine textured stoloniferous mat, excellent ground coverage, "
            "tight formation, no visible soil, research quality, "
            "natural lighting, top-down agricultural documentation, "
            "professional pasture photography"
        ),
        'negative': (
            "sparse coverage, bare patches, coarse texture, "
            "ground level view, artificial turf, ornamental, "
            "decorative lawn, flowers, synthetic"
        ),
        'species': 'cynodon',
        'condition': 'condicao_premium'
    },
    
    'cynodon_qualidade_padrao': {
        'positive': (
            "aerial view of standard quality cynodon pasture, "
            "good carpet coverage with minor thin areas, "
            "mostly uniform green with natural color variation, "
            "stoloniferous grass with adequate density, "
            "scientific field documentation, overhead perspective, "
            "realistic pasture conditions"
        ),
        'negative': (
            "severely thin, mostly bare soil, dying grass, "
            "ground view, decorative setting, artificial, "
            "ornamental turf, flowers, synthetic"
        ),
        'species': 'cynodon',
        'condition': 'qualidade_padrao'
    },
    
    'cynodon_cobertura_rarefeita': {
        'positive': (
            "top-down view of cynodon with thinning coverage, "
            "patchy grass areas with visible thin spots, "
            "mixed density stoloniferous growth, some soil showing, "
            "declining but still recognizable cynodon pasture, "
            "natural field conditions, research documentation, "
            "aerial agricultural survey"
        ),
        'negative': (
            "thick perfect carpet, uniform coverage, decorative turf, "
            "ground perspective, abstract patterns, unrecognizable, "
            "artificial, ornamental, flowers"
        ),
        'species': 'cynodon',
        'condition': 'cobertura_rarefeita'
    }
}

print(f"✓ Loaded {len(grass_prompts)} prompt configurations")
print(f"✓ Species: {len(set(p['species'] for p in grass_prompts.values()))}")
print(f"✓ Total combinations ready for mass generation")

In [ ]:
# Mass generation configuration
IMAGES_PER_TYPE = 100  # 100 images per type (species + condition)
BATCH_SIZE = 5         # Process 5 images at a time to optimize A100 usage
SAVE_EVERY = 10        # Save to Drive every 10 images to prevent loss

# Create execution timestamp for this run
execution_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
date_folder = datetime.now().strftime("%Y-%m-%d")

print(f"Mass Generation Configuration:")
print(f"  Images per type: {IMAGES_PER_TYPE}")
print(f"  Total types: {len(grass_prompts)}")
print(f"  Total images: {IMAGES_PER_TYPE * len(grass_prompts)}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Save frequency: every {SAVE_EVERY} images")
print(f"  Execution ID: {execution_timestamp}")
print(f"  Date: {date_folder}")

In [ ]:
# Mass generation functions optimized for A100
def generate_single_image(prompt_data, image_num, total_images):
    """Generate a single image with random parameters"""
    
    # Random parameters for variety
    seed = random.randint(0, 2**32 - 1)
    guidance_scale = random.uniform(4.5, 7.0)
    num_steps = random.randint(28, 35)
    
    # Slight prompt variations for more diversity
    prompt_variations = [
        prompt_data['positive'],
        prompt_data['positive'] + ", natural field variation",
        prompt_data['positive'] + ", agricultural documentation",
        prompt_data['positive'] + ", scientific photography"
    ]
    
    selected_prompt = random.choice(prompt_variations)
    
    # Generate image
    with torch.inference_mode():
        generator = torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu")
        generator.manual_seed(seed)
        
        image = pipe(
            prompt=selected_prompt,
            negative_prompt=prompt_data['negative'],
            num_inference_steps=num_steps,
            guidance_scale=guidance_scale,
            height=1024,
            width=1024,
            generator=generator
        ).images[0]
    
    return image, seed, guidance_scale, num_steps

def save_image_to_drive(image, species, condition, execution_timestamp, image_num, seed):
    """Save image directly to Google Drive with organized structure"""
    
    # Create organized directory structure
    species_dir = os.path.join(base_drive_dir, species, date_folder, f"execution_{execution_timestamp}")
    os.makedirs(species_dir, exist_ok=True)
    
    # Generate filename with seed for reproducibility
    filename = f"{species}_{condition}_{execution_timestamp}_{image_num:03d}_seed{seed}.png"
    filepath = os.path.join(species_dir, filename)
    
    # Save image
    image.save(filepath, "PNG")
    
    return filepath

def cleanup_memory():
    """Clean up GPU memory"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("✓ Mass generation functions loaded")

In [ ]:
# Main mass generation loop
print("🚀 Starting mass generation...")
print(f"Target: {IMAGES_PER_TYPE * len(grass_prompts)} total images")
print("="*60)

total_generated = 0
generation_log = []

# Process each prompt type
for prompt_key, prompt_data in grass_prompts.items():
    species = prompt_data['species']
    condition = prompt_data['condition']
    
    print(f"\n🌱 Generating {IMAGES_PER_TYPE} images for: {species}_{condition}")
    print(f"Prompt: {prompt_data['positive'][:80]}...")
    
    type_start_time = time.time()
    
    # Generate images for this type
    for img_num in tqdm(range(1, IMAGES_PER_TYPE + 1), desc=f"{species}_{condition}"):
        try:
            # Generate image
            image, seed, guidance, steps = generate_single_image(prompt_data, img_num, IMAGES_PER_TYPE)
            
            # Save directly to Drive
            filepath = save_image_to_drive(image, species, condition, execution_timestamp, img_num, seed)
            
            # Log generation details
            generation_log.append({
                'species': species,
                'condition': condition,
                'image_num': img_num,
                'seed': seed,
                'guidance_scale': guidance,
                'steps': steps,
                'filepath': filepath,
                'timestamp': datetime.now()
            })
            
            total_generated += 1
            
            # Periodic cleanup and progress update
            if img_num % SAVE_EVERY == 0:
                cleanup_memory()
                elapsed = time.time() - type_start_time
                avg_time = elapsed / img_num
                remaining = (IMAGES_PER_TYPE - img_num) * avg_time
                print(f"  Progress: {img_num}/{IMAGES_PER_TYPE} | Avg: {avg_time:.1f}s/img | ETA: {remaining/60:.1f}min")
            
        except Exception as e:
            print(f"  ❌ Error generating image {img_num}: {e}")
            cleanup_memory()
            continue
    
    type_duration = time.time() - type_start_time
    print(f"  ✅ Completed {species}_{condition} in {type_duration/60:.1f} minutes")
    print(f"  💾 All images saved to: {species}/{date_folder}/execution_{execution_timestamp}/")
    
    # Final cleanup for this type
    cleanup_memory()

print(f"\n🎉 MASS GENERATION COMPLETE!")
print(f"📊 Total images generated: {total_generated}")
print(f"📁 Saved to: {base_drive_dir}")
print(f"⏱️ Execution ID: {execution_timestamp}")

In [ ]:
# Generate detailed summary report
print("\n" + "="*80)
print("📋 DETAILED GENERATION SUMMARY")
print("="*80)

# Summary by species
species_summary = {}
for log_entry in generation_log:
    species = log_entry['species']
    if species not in species_summary:
        species_summary[species] = {'conditions': {}, 'total': 0}
    
    condition = log_entry['condition']
    if condition not in species_summary[species]['conditions']:
        species_summary[species]['conditions'][condition] = 0
    
    species_summary[species]['conditions'][condition] += 1
    species_summary[species]['total'] += 1

# Display summary
print(f"🕐 Generation completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🆔 Execution ID: {execution_timestamp}")
print(f"📅 Date folder: {date_folder}")
print(f"📊 Total images generated: {len(generation_log)}")
print(f"📁 Base directory: {base_drive_dir}")

print(f"\n📈 Generation breakdown by species:")
for species, data in species_summary.items():
    print(f"\n🌱 {species.upper()}:")
    print(f"   Total images: {data['total']}")
    for condition, count in data['conditions'].items():
        print(f"   - {condition}: {count} images")
        
        # Show folder structure
        folder_path = f"{species}/{date_folder}/execution_{execution_timestamp}/"
        print(f"     📂 {folder_path}")

# Generation statistics
if generation_log:
    seeds_used = [entry['seed'] for entry in generation_log]
    guidance_scales = [entry['guidance_scale'] for entry in generation_log]
    steps_used = [entry['steps'] for entry in generation_log]
    
    print(f"\n📊 Generation parameters statistics:")
    print(f"   Seeds range: {min(seeds_used)} - {max(seeds_used)}")
    print(f"   Guidance scale range: {min(guidance_scales):.1f} - {max(guidance_scales):.1f}")
    print(f"   Steps range: {min(steps_used)} - {max(steps_used)}")
    print(f"   Unique seeds used: {len(set(seeds_used))}")

print(f"\n💾 All images with organized structure and metadata saved to Google Drive!")
print(f"🔍 Each filename includes seed for reproducibility")
print("="*80)

In [ ]:
# Save detailed generation log to Google Drive
import json

# Prepare log for JSON serialization
serializable_log = []
for entry in generation_log:
    log_entry = entry.copy()
    log_entry['timestamp'] = entry['timestamp'].isoformat()
    serializable_log.append(log_entry)

# Save generation log
log_filename = f"generation_log_{execution_timestamp}.json"
log_filepath = os.path.join(base_drive_dir, log_filename)

with open(log_filepath, 'w') as f:
    json.dump({
        'execution_id': execution_timestamp,
        'date': date_folder,
        'total_images': len(generation_log),
        'images_per_type': IMAGES_PER_TYPE,
        'configuration': {
            'batch_size': BATCH_SIZE,
            'save_frequency': SAVE_EVERY,
            'model': 'stabilityai/stable-diffusion-3.5-medium'
        },
        'species_summary': species_summary,
        'detailed_log': serializable_log
    }, indent=2)

print(f"📋 Generation log saved: {log_filepath}")
print(f"🔍 This log contains all seeds and parameters for reproducibility")

# Create summary text file for easy reading
summary_filename = f"summary_{execution_timestamp}.txt"
summary_filepath = os.path.join(base_drive_dir, summary_filename)

with open(summary_filepath, 'w') as f:
    f.write(f"GRASS SPECIES MASS GENERATION SUMMARY\n")
    f.write(f"====================================\n\n")
    f.write(f"Execution ID: {execution_timestamp}\n")
    f.write(f"Date: {date_folder}\n")
    f.write(f"Total Images: {len(generation_log)}\n")
    f.write(f"Images per Type: {IMAGES_PER_TYPE}\n\n")
    
    f.write(f"SPECIES BREAKDOWN:\n")
    f.write(f"-----------------\n")
    for species, data in species_summary.items():
        f.write(f"\n{species.upper()}:\n")
        f.write(f"  Total: {data['total']} images\n")
        for condition, count in data['conditions'].items():
            f.write(f"  - {condition}: {count} images\n")
    
    f.write(f"\nFOLDER STRUCTURE:\n")
    f.write(f"----------------\n")
    f.write(f"grass_mass_generation/\n")
    for species in species_summary.keys():
        f.write(f"├── {species}/\n")
        f.write(f"│   └── {date_folder}/\n")
        f.write(f"│       └── execution_{execution_timestamp}/\n")
        f.write(f"│           └── {IMAGES_PER_TYPE * len(species_summary[species]['conditions'])} images\n")

print(f"📄 Summary text file saved: {summary_filepath}")
print(f"\n✅ All files and logs saved to Google Drive!")

In [ ]:
# Display sample images from generation
print("🖼️ Displaying sample images from generation...")

# Get samples from each species/condition combination
sample_images = {}
for species in species_summary.keys():
    for condition in species_summary[species]['conditions'].keys():
        # Find first image file for this combination
        species_dir = os.path.join(base_drive_dir, species, date_folder, f"execution_{execution_timestamp}")
        
        if os.path.exists(species_dir):
            files = [f for f in os.listdir(species_dir) if f.startswith(f"{species}_{condition}") and f.endswith('.png')]
            if files:
                sample_path = os.path.join(species_dir, files[0])
                try:
                    sample_img = Image.open(sample_path)
                    sample_images[f"{species}_{condition}"] = sample_img
                except Exception as e:
                    print(f"Could not load sample for {species}_{condition}: {e}")

# Display samples in grid
if sample_images:
    num_samples = len(sample_images)
    cols = 3
    rows = (num_samples + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    fig.suptitle(f"Sample Images from Mass Generation\nExecution: {execution_timestamp}", fontsize=16)
    
    for i, (name, img) in enumerate(sample_images.items()):
        row = i // cols
        col = i % cols
        
        axes[row, col].imshow(img)
        axes[row, col].set_title(name.replace('_', ' ').title(), fontsize=12)
        axes[row, col].axis('off')
    
    # Hide unused subplots
    for i in range(num_samples, rows * cols):
        row = i // cols
        col = i % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    
    # Save sample grid to Drive
    grid_filename = f"sample_grid_{execution_timestamp}.png"
    grid_filepath = os.path.join(base_drive_dir, grid_filename)
    plt.savefig(grid_filepath, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📸 Sample grid saved: {grid_filepath}")
else:
    print("⚠️ No sample images could be loaded")

print(f"\n🎉 Mass generation process complete!")
print(f"📁 Check your Google Drive: {base_drive_dir}")
print(f"🔍 All images have unique seeds for maximum diversity")